In [1]:
#Problem Set 5 :
# By Preethi Susan Abraham

import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso


import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

# import mglearn
from sklearn import tree

# Tree based models
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

# Kernel ridge
from sklearn.kernel_ridge import KernelRidge

# For timing: if you are interested 
from time import perf_counter

# Usual cross-val tools
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit
from sklearn.pipeline import make_pipeline

# Grid searching stuff
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit

In [2]:
bikedata = pd.read_csv("C:/Users/Preethi Abraham/Desktop/Brandeis Studies/Machine Learning and Data Analytics/bikeShareHour.csv")
# Checking for missing values (found none in any method)
bikedata.info()
bikedata.isnull()
bikedata.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     17379 non-null  int64  
 1   dteday      17379 non-null  object 
 2   season      17379 non-null  int64  
 3   yr          17379 non-null  int64  
 4   mnth        17379 non-null  int64  
 5   hr          17379 non-null  int64  
 6   holiday     17379 non-null  int64  
 7   weekday     17379 non-null  int64  
 8   workingday  17379 non-null  int64  
 9   weathersit  17379 non-null  int64  
 10  temp        17379 non-null  float64
 11  atemp       17379 non-null  float64
 12  hum         17379 non-null  float64
 13  windspeed   17379 non-null  float64
 14  casual      17379 non-null  int64  
 15  registered  17379 non-null  int64  
 16  cnt         17379 non-null  int64  
dtypes: float64(4), int64(12), object(1)
memory usage: 2.3+ MB


instant       0
dteday        0
season        0
yr            0
mnth          0
hr            0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

Bike share data info¶
- dteday : date
- season : season (1:springer, 2:summer, 3:fall, 4:winter)
- yr : year (0: 2011, 1:2012)
- mnth : month ( 1 to 12)
- hr : hour (0 to 23)
- holiday : weather day is holiday or not (extracted from http://dchr.dc.gov/page/holiday-schedule)
- weekday : day of the week
- workingday : if day is neither weekend nor holiday is 1, otherwise is 0.
+ weathersit : 
    - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
    - 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
    - 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
    - 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
- temp : Normalized temperature in Celsius. The values are divided to 41 (max)
- atemp: Normalized feeling temperature in Celsius. The values are divided to 50 (max)
- hum: Normalized humidity. The values are divided to 100 (max)
- windspeed: Normalized wind speed. The values are divided to 67 (max)
- casual: count of casual users
- registered: count of registered users
- cnt: count of total rental bikes including both casual and registered

In [3]:
# Create a peak hour dummy
# This clever code converts the boolean to an integer by multiplying by 1
bikedata['peakDummy'] = 1*(((bikedata.hr>=7) & (bikedata.hr<=9))|((bikedata.hr>=17)&(bikedata.hr<=19)))

In [4]:
# This creates data set of predictors that you should use
X = bikedata.filter(['yr', 'hr', 'season', 'holiday', 'weekday', 'workingday', 'weathersit', 'atemp', 'windspeed', 'WeekendDummy', 'peakDummy'], axis=1)
y = bikedata['cnt']
# Reduce size of dataset
X = X[:5000]
y = y[:5000]
print(X.shape)
X.head()

(5000, 10)


,yr,hr,season,holiday,weekday,workingday,weathersit,atemp,windspeed,peakDummy
0,0,0,1,0,6,0,1,0.2879,0.0,0
1,0,1,1,0,6,0,1,0.2727,0.0,0
2,0,2,1,0,6,0,1,0.2727,0.0,0
3,0,3,1,0,6,0,1,0.2879,0.0,0
4,0,4,1,0,6,0,1,0.2879,0.0,0


In [5]:
X.describe()

,yr,hr,season,holiday,weekday,workingday,weathersit,atemp,windspeed,peakDummy
count,5000.0,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000
mean,0.0,11.601400,1.836600,0.024000,3.002800,0.680600,1.41840,0.459816,0.204327,0.252800
std,0.0,6.909824,0.729936,0.153064,2.019804,0.466291,0.63547,0.198876,0.123378,0.434661
min,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,1.00000,0.000000,0.000000,0.000000
25%,0.0,6.000000,1.000000,0.000000,1.000000,0.000000,1.00000,0.287900,0.104500,0.000000
50%,0.0,12.000000,2.000000,0.000000,3.000000,1.000000,1.00000,0.469700,0.194000,0.000000
75%,0.0,18.000000,2.000000,0.000000,5.000000,1.000000,2.00000,0.621200,0.283600,1.000000
max,0.0,23.000000,3.000000,1.000000,6.000000,1.000000,4.00000,1.000000,0.850700,1.000000


In [6]:
# Linear regression
# Linear regression
nmc = 25
shuffle = ShuffleSplit(n_splits=nmc, test_size=.2)
lrm = LinearRegression()
CVInfo = cross_validate(lrm, X, y, cv=shuffle,return_train_score=True,n_jobs=1)
print(np.mean(CVInfo['train_score']))
print(np.mean(CVInfo['test_score']))

0.5476753680026929
0.5396997924598971


For this problem you should use the bike sharing data set which is given below.  Also, use the starter notebook to clean and organize the data from the csv file.  This notebook also limits the data to the first 5000 instances (out of about 17,000).  This is done for speed and memory usage issues.  This data covers bike sharing activity in Washington DC.  Beyond cleaning, the starter file gives a simple linear regression too.

For this problem we will look at the performance of three different models beyond linear regression.  In each case should should to a gridSearchCV, and use randomized cross validation to evaluate the different models. In each case use 25 randomized draws of training and test sets.  Set the test size to 0.2 of the sample.  All are performed on the X and y data array set up in the starter.  Model performance is judged by mean R-squared evaluated across the test samples.

In each case also run a standard randomized cross validation test for 50 iterations at your optimal parameters from the search.

Note:  All of these should run in less than 30 minutes.  Probably most will take under 15 minutes.  The slowest is the kernel ridge regression.  It also is the method that uses the most memory.  This is actually consistent with some timing results on kernel based methods which suggest they are not great for large sample sizes.

1) Evaluate kernel ridge regression using alpha = [0.01,0.05,0.25, 0.5] and gamma = [0.01,0.05,0.25,0.5], and rbf kernel.  For this case make sure you apply the standard scaler to your data (use a pipeline).

2) Evaluate a regression tree with max_depth equal to [5,10,15,20,25,50].

3) Evaluate a random forest with max_depth equal to [5,10,15,20,25], max_features = [3,5,10], and n_estimators = [100].  (If you computer is very fast, you might try other numbers of trees of 10, and 25 just to check.  In solutions I will add a bunch more of these from my big machine.)

4.) Which ended up being the best model, and best hyperparameter combination?

5.) Why didn't you need to use the standardScaler in parts 2 and 3?

In [15]:
#1.  Evaluate kernel ridge regression using alpha = [0.01,0.05,0.25, 0.5] and gamma = [0.01,0.05,0.25,0.5], and rbf kernel.  For this case make sure you apply the standard scaler to your data (use a pipeline).

#For 25 splits
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

score_used = 'r2'
fullModel = Pipeline([
    ("scaler", StandardScaler()),
    ("kridge", KernelRidge())
])

param_grid={'kridge__alpha':[0.01,0.05,0.25, 0.5],'kridge__gamma':[0.01,0.05,0.25,0.5],'kridge__kernel':['rbf']}
shuffle_split = ShuffleSplit(test_size=0.2, n_splits=25)
grid_search=GridSearchCV(fullModel,param_grid,cv=shuffle_split,scoring=score_used, 
                              return_train_score=True,n_jobs=1)
grid_search.fit(X,y)
results = pd.DataFrame(grid_search.cv_results_)
print(results[['rank_test_score','mean_test_score','param_kridge__alpha','param_kridge__gamma']])

    rank_test_score  mean_test_score param_kridge__alpha param_kridge__gamma
0                13         0.796639                0.01                0.01
1                 6         0.861405                0.01                0.05
2                 5         0.864318                0.01                0.25
3                10         0.841212                0.01                 0.5
4                14         0.763394                0.05                0.01
5                 9         0.849969                0.05                0.05
6                 1         0.871705                0.05                0.25
7                 4         0.865025                0.05                 0.5
8                15         0.734064                0.25                0.01
9                11         0.823873                0.25                0.05
10                3         0.866076                0.25                0.25
11                2         0.866284                0.25                 0.5

In [7]:
#1. Evaluate kernel ridge regression using alpha = [0.01,0.05,0.25, 0.5] and gamma = [0.01,0.05,0.25,0.5], and rbf kernel.  For this case make sure you apply the standard scaler to your data (use a pipeline).


#For 50 splits
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

score_used = 'r2'
fullModel = Pipeline([
    ("scaler", StandardScaler()),
    ("kridge", KernelRidge())
])

param_grid={'kridge__alpha':[0.01,0.05,0.25, 0.5],'kridge__gamma':[0.01,0.05,0.25,0.5],'kridge__kernel':['rbf']}
shuffle_split = ShuffleSplit(test_size=0.2, n_splits=50)
grid_search=GridSearchCV(fullModel,param_grid,cv=shuffle_split,scoring=score_used, 
                              return_train_score=True,n_jobs=1)
grid_search.fit(X,y)
results = pd.DataFrame(grid_search.cv_results_)
print(results[['rank_test_score','mean_test_score','param_kridge__alpha','param_kridge__gamma']])

    rank_test_score  mean_test_score param_kridge__alpha param_kridge__gamma
0                13         0.794461                0.01                0.01
1                 6         0.860253                0.01                0.05
2                 4         0.862254                0.01                0.25
3                10         0.833931                0.01                 0.5
4                14         0.761181                0.05                0.01
5                 9         0.848283                0.05                0.05
6                 1         0.870591                0.05                0.25
7                 5         0.861188                0.05                 0.5
8                15         0.732065                0.25                0.01
9                11         0.821896                0.25                0.05
10                2         0.864298                0.25                0.25
11                3         0.863040                0.25                 0.5

In [8]:
# standard randomized cross validation test
CVInfo = cross_validate(fullModel, X, y, cv=shuffle,return_train_score=True,n_jobs=1)
print(np.mean(CVInfo['train_score']))
print(np.mean(CVInfo['test_score']))

-0.5136933829244331
-0.5189331342799605


In [16]:
#2.Evaluate a regression tree with max_depth equal to [5,10,15,20,25,50].

#For 25 splits
from sklearn.tree import DecisionTreeRegressor
score_used = 'r2' 
param_grid={'max_depth':[5,10,15,20,25,50]}
shuffle_split = ShuffleSplit(test_size=0.2, n_splits=25)
grid_search=GridSearchCV(DecisionTreeRegressor(),param_grid,cv=shuffle_split,scoring=score_used, 
                              return_train_score=True)
grid_search.fit(X,y)
results = pd.DataFrame(grid_search.cv_results_)
print(results[['rank_test_score','param_max_depth','mean_test_score']])

   rank_test_score param_max_depth  mean_test_score
0                6               5         0.792745
1                1              10         0.896364
2                2              15         0.878976
3                4              20         0.875133
4                5              25         0.874288
5                3              50         0.875319


In [9]:
#For 50 splits

from sklearn.tree import DecisionTreeRegressor
score_used = 'r2' 
param_grid={'max_depth':[5,10,15,20,25,50]}
shuffle_split = ShuffleSplit(test_size=0.2, n_splits=50)
grid_search=GridSearchCV(DecisionTreeRegressor(),param_grid,cv=shuffle_split,scoring=score_used, 
                              return_train_score=True)
grid_search.fit(X,y)


GridSearchCV(cv=ShuffleSplit(n_splits=50, random_state=None, test_size=0.2, train_size=None),
             error_score=nan,
             estimator=DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse',
                                             max_depth=None, max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort='deprecated',
                                             random_state=None,
                                             splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'max_depth': [5, 10, 15, 20, 25, 50]},
   

In [10]:
results = pd.DataFrame(grid_search.cv_results_)
print(results[['rank_test_score','param_max_depth','mean_test_score']])

   rank_test_score param_max_depth  mean_test_score
0                6               5         0.794169
1                1              10         0.897474
2                2              15         0.883121
3                4              20         0.877782
4                3              25         0.878152
5                5              50         0.877036


In [11]:
# standard randomized cross validation test
dtr=DecisionTreeRegressor(max_depth=10)
CVInfo = cross_validate(dtr, X, y, cv=shuffle,return_train_score=True,n_jobs=1)
print(np.mean(CVInfo['train_score']))
print(np.mean(CVInfo['test_score']))

0.9563839036052764
0.8996801929454334


In [17]:
#3.Evaluate a random forest with max_depth equal to [5,10,15,20,25], max_features = [3,5,10], and n_estimators = [100].  (If you computer is very fast, you might try other numbers of trees of 10, and 25 just to check.  In solutions I will add a bunch more of these from my big machine.)

#For 25 splits
core_used = 'r2' 
param_grid={'n_estimators':[100],'max_depth':[5,10,15,20,25],'max_features':[3,5,10]}
shuffle_split = ShuffleSplit(test_size=0.2, n_splits=25)
grid_search=GridSearchCV(RandomForestRegressor(),param_grid,cv=shuffle_split,scoring=score_used, 
                              return_train_score=True)
grid_search.fit(X,y)
results = pd.DataFrame(grid_search.cv_results_)
print(results[['rank_test_score','mean_test_score','param_max_depth','param_n_estimators','param_max_features']])

    rank_test_score  mean_test_score param_max_depth param_n_estimators  \
0                15         0.746198               5                100   
1                14         0.791701               5                100   
2                13         0.807929               5                100   
3                12         0.897967              10                100   
4                 8         0.923336              10                100   
5                 7         0.927492              10                100   
6                11         0.922125              15                100   
7                 2         0.933475              15                100   
8                 4         0.930712              15                100   
9                10         0.922694              20                100   
10                3         0.933313              20                100   
11                6         0.929852              20                100   
12                9      

In [12]:
#For 50 splits
core_used = 'r2' 
param_grid={'n_estimators':[100],'max_depth':[5,10,15,20,25],'max_features':[3,5,10]}
shuffle_split = ShuffleSplit(test_size=0.2, n_splits=50)
grid_search=GridSearchCV(RandomForestRegressor(),param_grid,cv=shuffle_split,scoring=score_used, 
                              return_train_score=True)
grid_search.fit(X,y)
results = pd.DataFrame(grid_search.cv_results_)
print(results[['rank_test_score','mean_test_score','param_max_depth','param_n_estimators','param_max_features']])

    rank_test_score  mean_test_score param_max_depth param_n_estimators  \
0                15         0.747533               5                100   
1                14         0.791692               5                100   
2                13         0.809869               5                100   
3                12         0.897781              10                100   
4                 8         0.923555              10                100   
5                 7         0.928781              10                100   
6                11         0.922114              15                100   
7                 2         0.934196              15                100   
8                 4         0.931448              15                100   
9                10         0.923061              20                100   
10                3         0.933932              20                100   
11                5         0.931296              20                100   
12                9      

In [13]:
# standard randomized cross validation test
rfr=RandomForestRegressor(n_estimators=100,max_depth=15)
CVInfo = cross_validate(rfr, X, y, cv=shuffle,return_train_score=True,n_jobs=1)
print(np.mean(CVInfo['train_score']))
print(np.mean(CVInfo['test_score']))

0.987984739201689
0.9325268122839403


### 4. Which ended up being the best model, and best hyperparameter combination?

The RandomForestRegressor() model is the best model. The mean test score is the highest (0.934228) when hyperparameters are set to max_depth= 25, n_estimators=100 and max_features=5


### 5. Why didn't you need to use the standardScaler in parts 2 and 3?

The Regression Tree (Part 2) and Random Forest Regression (Part 3) algorithms do not need the features to be scaled. This is because these algorithms are not sensitive to the magnitude of the features. However, the features need to be scaled (standardized) when we use Kernel Ridge Regression (Part 1). This is because this algorithm will penalize the model based on the magnitude of the feature variables. If the variance of a feature is large, then the coefficients obtained from the kernel ridge regression will be small. As a result, the penalty will be low.